# Neural Machine Translation with Attention

If you wanna read more about this type of Machine Translation, [Wikipedia](https://en.wikipedia.org/wiki/Neural_machine_translation) is a good source.

This notebook has been inspited by the Deeplearning.ai - [Sequence models course](https://www.coursera.org/learn/nlp-sequence-models) - Attention mechanism.

The idea is to build a Neural Machine Translation (NMT) model to translate human readable dates ("10th of September, 1978") into machine readable dates ("1978-09-10"), using attention model.

### Attention mechanism
If you had to translate a book's paragraph from French to English, you would not read the whole paragraph, then close the book and translate. Even during the translation process, you would read/re-read and focus on the parts of the French paragraph corresponding to the parts of the English you are writing down.

The attention mechanism tells a Neural Machine Translation model where it should pay attention to at any step. [Andrew Ng](http://www.andrewng.org/) explains this mechanism quite well in these tho videos: [Attention model intuition](https://www.youtube.com/watch?v=EgCPRvgltoE) and [Attention model](https://www.youtube.com/watch?v=ahZRxHaM7gU)
 
Here is a figure to remind you how the model works. The diagram on the left shows the attention model:

<table>
    <tbody>
        <tr>
            <td><img src="attn_model.png" style="height:500px" /></td>
            <td><img src="attn_mechanism.png" style="height:500px" /></td>
        </tr>
    </tbody>
</table>

Here are some properties of the model: 

- There are two separate LSTMs in this model (see diagram on the left). Because the one at the bottom of the picture is a Bi-directional LSTM and comes *before* the attention mechanism, we will call it *pre-attention* Bi-LSTM. The LSTM at the top of the diagram comes *after* the attention mechanism, so we will call it the *post-attention* LSTM. The pre-attention Bi-LSTM goes through $T_x$ time steps; the post-attention LSTM goes through $T_y$ time steps. 

- The post-attention LSTM passes $s^{\langle t \rangle}, c^{\langle t \rangle}$ from one time step to the next. The LSTM has both the output activation $s^{\langle t\rangle}$ and the hidden cell state $c^{\langle t\rangle}$. In this model the post-activation LSTM at time $t$ does will not take the specific generated $y^{\langle t-1 \rangle}$ as input; it only takes $s^{\langle t\rangle}$ and $c^{\langle t\rangle}$ as input. We have designed the model this way, because (unlike language generation where adjacent characters are highly correlated) there isn't as strong a dependency between the previous character and the next character in a YYYY-MM-DD date. 

- We use $a^{\langle t \rangle} = [\overrightarrow{a}^{\langle t \rangle}; \overleftarrow{a}^{\langle t \rangle}]$ to represent the concatenation of the activations of both the forward-direction and backward-directions of the pre-attention Bi-LSTM. 

- The diagram on the right uses a `RepeatVector` node to copy $s^{\langle t-1 \rangle}$'s value $T_x$ times, and then `Concatenation` to concatenate $s^{\langle t-1 \rangle}$ and $a^{\langle t \rangle}$ to compute $e^{\langle t, t'}$, which is then passed through a softmax to compute $\alpha^{\langle t, t' \rangle}$. We'll explain how to use `RepeatVector` and `Concatenation` in Keras below. 

Let's begin importing the necessary packages:

In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np
from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


Following we'll use [Faker](https://pypi.org/project/Faker/) to generate our own dataset of human readable date with their iso format (machine readable date) as labels.

In [2]:
fake = Faker()

# We need to seed these guys. For some reason I always use 101
fake.seed(101)
random.seed(101)

We're gonna generate a dataset with different formats. In this case I'm giving more chances to the full format, since it's the most used one.

In [3]:
FORMATS = ['short',
           'medium',
           'long',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'd MMM YYY', 
           'd MMMM YYY',
           'dd MMM YYY',
           'dd MMMM, YYY',
           'MMM d YYY',
           'MMMM d YYY',
           'MMM dd YYY',
           'MMMM dd YYY',
           'd-MM-YY',
           'd-MM-YYY',
           'dd-MM-YY',
           'dd-MM-YYY',
           'EE d, MMM YYY',
           'EE dd, MMM YYY',
           'EEEE d, MMMM YYY',
           'EEEE dd, MMMM YYY']

Let's have a look at those formats:

In [4]:
for format in FORMATS:
    print('%s => %s' %(format, format_date(fake.date_object(), format=format, locale='en')))

short => 7/19/09
medium => Apr 3, 1983
long => September 11, 2006
full => Sunday, May 29, 1994
full => Monday, October 15, 2001
full => Friday, April 20, 1973
full => Tuesday, February 24, 2015
full => Wednesday, April 7, 2004
full => Monday, August 6, 1984
full => Monday, December 20, 2010
full => Friday, February 1, 1985
full => Sunday, August 20, 1989
full => Tuesday, January 14, 2003
d MMM YYY => 13 Feb 2017
d MMMM YYY => 14 June 1984
dd MMM YYY => 23 May 1992
dd MMMM, YYY => 22 October, 1999
MMM d YYY => Oct 15 1974
MMMM d YYY => May 16 1987
MMM dd YYY => Feb 26 1983
MMMM dd YYY => December 17 1980
d-MM-YY => 18-01-76
d-MM-YYY => 18-04-2000
dd-MM-YY => 26-07-06
dd-MM-YYY => 10-12-1994
EE d, MMM YYY => Sat 15, Jan 1983
EE dd, MMM YYY => Tue 05, Feb 2002
EEEE d, MMMM YYY => Monday 9, March 1992
EEEE dd, MMMM YYY => Wednesday 02, April 1986


`random_date()` will generate a random date using a random format picked from our list `FORMATS` defined before. It'll return a tuple with the human and machine readable date plus the date object:

In [5]:
def random_date():
    dt = fake.date_object()

    try:
        date = format_date(dt, format=random.choice(FORMATS), locale='en')
        human_readable = date.lower().replace(',', '')
        machine_readable = dt.isoformat()

    except AttributeError as e:
        return None, None, None

    return human_readable, machine_readable, dt

`create_dataset(m)` will generate our dataset, taking `m` as the number of samples to create. It returns the `dataset` as a list, two dictionaries mapping index to character (these are our vocabularies), `human` and `machine`, and the inverse mapping, `inv_machine`, chars to index:

In [6]:
def create_dataset(m):
    human_vocab = set()
    machine_vocab = set()
    dataset = []
    
    for i in tqdm(range(m)):
        h, m, _ = random_date()
        if h is not None:
            dataset.append((h, m))
            human_vocab.update(tuple(h))
            machine_vocab.update(tuple(m))
    
    # We also add two special chars, <unk> for unknown characters, and <pad> to add padding at the end
    human = dict(zip(sorted(human_vocab) + ['<unk>', '<pad>'], list(range(len(human_vocab) + 2))))
    inv_machine = dict(enumerate(sorted(machine_vocab)))
    machine = {v: k for k, v in inv_machine.items()}
 
    return dataset, human, machine, inv_machine

Let's go easy and generate a dataset with 10k samples. That should do a good job:

In [7]:
m = 10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = create_dataset(m)

100%|██████████| 10000/10000 [00:00<00:00, 14730.49it/s]


Inspecting the first 10 entries. Remember it contains a list of tuples => (human readable, machine readable):

In [8]:
dataset[:10]

[('may 18 2013', '2013-05-18'),
 ('thursday 26 june 1997', '1997-06-26'),
 ('sunday september 27 1998', '1998-09-27'),
 ('mar 4 1984', '1984-03-04'),
 ('saturday april 13 1974', '1974-04-13'),
 ('10 april 1999', '1999-04-10'),
 ('oct 30 2001', '2001-10-30'),
 ('16-09-90', '1990-09-16'),
 ('30 may 1984', '1984-05-30'),
 ('saturday july 16 1977', '1977-07-16')]

Let's have a look at our human readable vocabulary:

In [9]:
human_vocab

{' ': 0,
 '-': 1,
 '/': 2,
 '0': 3,
 '1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '7': 10,
 '8': 11,
 '9': 12,
 'a': 13,
 'b': 14,
 'c': 15,
 'd': 16,
 'e': 17,
 'f': 18,
 'g': 19,
 'h': 20,
 'i': 21,
 'j': 22,
 'l': 23,
 'm': 24,
 'n': 25,
 'o': 26,
 'p': 27,
 'r': 28,
 's': 29,
 't': 30,
 'u': 31,
 'v': 32,
 'w': 33,
 'y': 34,
 '<unk>': 35,
 '<pad>': 36}

Machine readable vocabulary:

In [10]:
machine_vocab

{'-': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10}

... and its inverse dictionary:

In [11]:
inv_machine_vocab

{0: '-',
 1: '0',
 2: '1',
 3: '2',
 4: '3',
 5: '4',
 6: '5',
 7: '6',
 8: '7',
 9: '8',
 10: '9'}

`preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)` is gonna do some beautiful magic with our dataset. It takes the whole dataset, and both human and machine vocabularie, plus some max length arguments, and it'll spit out out training set and target labels, plus the [one hot encoding](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f) of both:

In [12]:
def preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty):
    X, Y = zip(*dataset)
    
    X = np.array([string_to_int(i, Tx, human_vocab) for i in X])
    Y = [string_to_int(t, Ty, machine_vocab) for t in Y]
    
    Xoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), X)))
    Yoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), Y)))

    return X, np.array(Y), Xoh, Yoh

`string_to_int(string, length, vocab)` will return a list of indexes based on a `string` and vocabulary, `vocab`, cropping or padding it depending on the max `length` passed in:

In [13]:
def string_to_int(string, length, vocab):
    string = string.lower()
    string = string.replace(',','')
    
    if len(string) > length:
        string = string[:length]
        
    rep = list(map(lambda x: vocab.get(x, '<unk>'), string))
    
    if len(string) < length:
        rep += [vocab['<pad>']] * (length - len(string))
    
    return rep

Let's have a look at an example. By the way, that's my birthday 😉:

In [14]:
string_to_int('September 10, 1978', 30, human_vocab)

[29,
 17,
 27,
 30,
 17,
 24,
 14,
 17,
 28,
 0,
 4,
 3,
 0,
 4,
 12,
 10,
 11,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36]

Let's run the preprocessing and print out some shapes:

In [15]:
Tx = 30
Ty = 10
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (10000, 30)
Y.shape: (10000, 10)
Xoh.shape: (10000, 30, 37)
Yoh.shape: (10000, 10, 11)


... and see what a training sample, target label and their respective one hot encoding look like:

In [16]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

Source date: may 18 2013
Target date: 2013-05-18

Source after preprocessing (indices): [24 13 34  0  4 11  0  5  3  4  6 36 36 36 36 36 36 36 36 36 36 36 36 36
 36 36 36 36 36 36]
Target after preprocessing (indices): [3 1 2 4 0 1 6 0 2 9]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


This is what we have now:

- **X**: a processed version of the human readable dates in the training set, where each character is replaced by an index mapped to the character via human_vocab. Each date is further padded to `Tx` values with a special character `<pad>`. `X.shape = (m, Tx)`

- **Y**: a processed version of the machine readable dates in the training set, where each character is replaced by the index it is mapped to in machine_vocab. You should have `Y.shape = (m, Ty)`.

- **Xoh**: one-hot version of X, the "1" entry's index is mapped to the character thanks to human_vocab. `Xoh.shape = (m, Tx, len(human_vocab))`.

- **Yoh**: one-hot version of Y, the "1" entry's index is mapped to the character thanks to machine_vocab. `Yoh.shape = (m, Tx, len(machine_vocab))`. Here, `len(machine_vocab) = 11` since there are 11 characters ('-' as well as 0-9).

Let's define some layers we need as global variables. [RepeatVector()](https://keras.io/layers/core/#repeatvector), [Concatenate()](https://keras.io/layers/merge/#concatenate), [Dense()](https://keras.io/layers/core/#dense), [Activation()](https://keras.io/layers/core/#activation), [Dot()](https://keras.io/layers/merge/#dot)

In [17]:
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation('softmax', name='attention_weights')
dotor = Dot(axes = 1)

`one_step_attention(a, s_prev)`: At step $t$, given all the hidden states of the Bi-LSTM ($[a^{<1>},a^{<2>}, ..., a^{<T_x>}]$) and the previous hidden state of the second LSTM ($s^{<t-1>}$), `one_step_attention()` will compute the attention weights ($[\alpha^{<t,1>},\alpha^{<t,2>}, ..., \alpha^{<t,T_x>}]$) and output the context vector:
$$context^{<t>} = \sum_{t' = 0}^{T_x} \alpha^{<t,t'>}a^{<t'>}\tag{1}$$

In [18]:
def one_step_attention(a, s_prev):
    s_prev = repeator(s_prev)
    concat = concatenator([a, s_prev])
    e = densor1(concat)
    energies = densor2(e)
    alphas = activator(energies)
    context = dotor([alphas, a])
    
    return context

In [19]:
n_a = 32
n_s = 64
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation='softmax')

`model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size)`: Implements the entire model. It first runs the input through a [Bidirectional](https://keras.io/layers/wrappers/#bidirectional) [LSTM](https://keras.io/layers/recurrent/#lstm) to get back $[a^{<1>},a^{<2>}, ..., a^{<T_x>}]$. Then, it calls `one_step_attention()` $T_y$ times (`for` loop). At each iteration of this loop, it gives the computed context vector $c^{<t>}$ to the second LSTM, and runs the output of the LSTM through a dense layer with softmax activation to generate a prediction $\hat{y}^{<t>}$. 

In [20]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    outputs = []
    
    a = Bidirectional(LSTM(n_a, return_sequences = True))(X)
    
    for t in range(Ty):
        context = one_step_attention(a, s)
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        out = output_layer(s)
        outputs.append(out)
    
    model = Model([X, s0, c0], outputs)
    return model

Model instantiation and summary representation of the model:

In [21]:
mod = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

In [22]:
mod.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30, 37)       0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 30, 64)       17920       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 30, 64)       0           s0[0][0]                         
                                                                 lstm_1[0][0]                     
          

Using [Adam optimizer](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/) we proceed to compile and train our model:

In [23]:
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
mod.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

In [25]:
mod.fit([Xoh, s0, c0], outputs, epochs=30, batch_size=100)

Epoch 1/30
10000/10000 [==============================] - 23s 2ms/step - loss: 15.2894 - dense_3_loss: 2.4340 - dense_3_acc: 0.7520 - dense_3_acc_1: 0.8487 - dense_3_acc_2: 0.4732 - dense_3_acc_3: 0.1575 - dense_3_acc_4: 0.8064 - dense_3_acc_5: 0.3702 - dense_3_acc_6: 0.1315 - dense_3_acc_7: 0.7262 - dense_3_acc_8: 0.2698 - dense_3_acc_9: 0.1423
Epoch 2/30
10000/10000 [==============================] - 14s 1ms/step - loss: 7.3339 - dense_3_loss: 1.6648 - dense_3_acc: 0.9708 - dense_3_acc_1: 0.9708 - dense_3_acc_2: 0.7935 - dense_3_acc_3: 0.4494 - dense_3_acc_4: 0.9999 - dense_3_acc_5: 0.8416 - dense_3_acc_6: 0.4136 - dense_3_acc_7: 0.9995 - dense_3_acc_8: 0.5488 - dense_3_acc_9: 0.3529
Epoch 3/30
10000/10000 [==============================] - 14s 1ms/step - loss: 5.8312 - dense_3_loss: 1.4489 - dense_3_acc: 0.9766 - dense_3_acc_1: 0.9778 - dense_3_acc_2: 0.8449 - dense_3_acc_3: 0.5404 - dense_3_acc_4: 0.9999 - dense_3_acc_5: 0.9190 - dense_3_acc_6: 0.5433 - dense_3_acc_7: 0.9998 - dens

10000/10000 [==============================] - 14s 1ms/step - loss: 1.2471 - dense_3_loss: 0.3063 - dense_3_acc: 0.9875 - dense_3_acc_1: 0.9889 - dense_3_acc_2: 0.9265 - dense_3_acc_3: 0.9699 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9847 - dense_3_acc_6: 0.9232 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9124 - dense_3_acc_9: 0.8998
Epoch 25/30
10000/10000 [==============================] - 14s 1ms/step - loss: 1.2160 - dense_3_loss: 0.2994 - dense_3_acc: 0.9872 - dense_3_acc_1: 0.9887 - dense_3_acc_2: 0.9313 - dense_3_acc_3: 0.9721 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9846 - dense_3_acc_6: 0.9252 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9159 - dense_3_acc_9: 0.9025
Epoch 26/30
10000/10000 [==============================] - 14s 1ms/step - loss: 1.1765 - dense_3_loss: 0.2889 - dense_3_acc: 0.9875 - dense_3_acc_1: 0.9900 - dense_3_acc_2: 0.9336 - dense_3_acc_3: 0.9736 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9846 - dense_3_acc_6: 0.9277 - dense_3_acc_7: 1.0000 - dense_3_acc_8:

Let's make some predictions to see whether it gets them right:

In [26]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
for example in EXAMPLES:
    
    source = string_to_int(example, Tx, human_vocab)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source)))
    source = source.reshape((1, ) + source.shape)
    prediction = mod.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    
    print("source:", example)
    print("output:", ''.join(output))

source: 3 May 1979
output: 1999-05-03
source: 5 April 09
output: 2009-04-05
source: 21th of August 2016
output: 2016-08-21
source: Tue 10 Jul 2007
output: 2007-07-11
source: Saturday May 9 2018
output: 2018-05-09
source: March 3 2001
output: 2011-03-03
source: March 3rd 2001
output: 2011-03-03
source: 1 March 2001
output: 2011-03-01


Finally we save the model to be converted and used by our Frontend component using [TensorFlow.js](https://js.tensorflow.org/)

In [27]:
mod.save('dates_model.h5')

/usr/local/lib/python3.6/site-packages/keras/engine/network.py:872: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 's0:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'c0:0' shape=(?, 64) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/usr/local/lib/python3.6/site-packages/keras/engine/network.py:872: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/TensorArrayReadV3:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_4:0' shape=(?, 64) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/usr/local/lib/python3.6/site-packages/keras/engine/network.py:872: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lst